In [3]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
from matplotlib import pyplot as plt
import pandas as pd
import glob
import numpy as np

In [4]:
files = glob.glob('C:\D\SKRIPSI\yolov5\\runs\detect\inference_10\labels\*.txt')

data = [pd.read_csv(file, sep=" ", names=['class','x', 'y', 'w','h']) for file in files]

In [5]:
data

[   class         x         y         w         h
 0      0  0.704687  0.987500  0.040625  0.025000
 1      0  0.510938  0.125000  0.031250  0.065625
 2      0  0.544531  0.277344  0.042188  0.092188
 3      0  0.078125  0.910156  0.078125  0.179688
 4      0  0.615625  0.575781  0.068750  0.160938
 5      0  0.167187  0.470313  0.053125  0.134375,
    class         x         y         w         h
 0      0  0.513281  0.171094  0.039062  0.085938
 1      0  0.560938  0.360938  0.050000  0.109375
 2      0  0.194531  0.312500  0.042188  0.100000
 3      0  0.661719  0.745313  0.089063  0.212500
 4      0  0.145313  0.607031  0.062500  0.173438,
    class         x         y         w         h
 0      0  0.039844  0.594531  0.060937  0.117188
 1      0  0.174219  0.423438  0.045312  0.118750
 2      0  0.528125  0.242969  0.046875  0.098437
 3      0  0.729688  0.938281  0.084375  0.123438
 4      0  0.596094  0.485156  0.067187  0.145313
 5      0  0.119531  0.832031  0.064062  0.22968

In [6]:
df = pd.concat(data, ignore_index=True)

In [7]:
df

,class,x,y,w,h
0,0,0.704687,0.987500,0.040625,0.025000
1,0,0.510938,0.125000,0.031250,0.065625
2,0,0.544531,0.277344,0.042188,0.092188
3,0,0.078125,0.910156,0.078125,0.179688
4,0,0.615625,0.575781,0.068750,0.160938
...,...,...,...,...,...
856,0,0.709375,0.893750,0.075000,0.106250
857,0,0.303125,0.382812,0.037500,0.100000
858,0,0.319531,0.443750,0.045312,0.121875
859,0,0.517187,0.128906,0.081250,0.104687


In [9]:
cols = list(df)[1:5]

In [10]:
raw_df = df[cols].astype(float)

In [11]:
raw_df

,x,y,w,h
0,0.704687,0.987500,0.040625,0.025000
1,0.510938,0.125000,0.031250,0.065625
2,0.544531,0.277344,0.042188,0.092188
3,0.078125,0.910156,0.078125,0.179688
4,0.615625,0.575781,0.068750,0.160938
...,...,...,...,...
856,0.709375,0.893750,0.075000,0.106250
857,0.303125,0.382812,0.037500,0.100000
858,0.319531,0.443750,0.045312,0.121875
859,0.517187,0.128906,0.081250,0.104687


In [13]:
raw_df['center_x'] = (raw_df['x'] + raw_df['w']) / 2
raw_df['center_y'] = (raw_df['y'] + raw_df['h']) / 2

In [18]:
import tensorflow as tf
import keras_tuner as kt
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf1
from keras.layers import Input

In [153]:
feature_df = raw_df[['center_x', 'center_y']].values

In [150]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [155]:
feature_scaler = scaler.fit_transform(feature_df).reshape(1, feature_df.shape[0], feature_df.shape[1])


In [160]:
feature_scaler

array([[[0.6897315 , 0.8845615 ],
        [0.49628012, 0.13951847],
        [0.53869011, 0.30170012],
        ...,
        [0.32738064, 0.47946206],
        [0.54985058, 0.17846966],
        [0.34672635, 0.55524138]]])

In [171]:
# split_point = int(len(feature_scaler) * 0.8)
# train = feature_scaler[:split_point]
# test = feature_scaler[split_point:]
from sklearn.model_selection import train_test_split

# Create a sample array with shape (1, 861, 2)

# Reshape the array to (861, 2) for splitting
feature_arr = feature_scaler.reshape(861, 2)

# Split the data into train and test sets
train_data, test_data = train_test_split(feature_arr, test_size=0.2)

# Reshape the split data back to (1, n, 2)
train_data = train_data.reshape(1, -1, 2)
test_data = test_data.reshape(1, -1, 2)

In [20]:
class LSTMHyperModel(kt.HyperModel):
    def build(self, hp):
        model = Sequential()
        model.add(Input(shape=(None, 2)))  # Adjust input shape based on your data

        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(LSTM(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                                  return_sequences=True if i < hp.Int('num_layers', 1, 3) - 1 else False))
        
        model.add(Dense(2))  # Output layer with 2 units for center_x and center_y

        model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                      loss='mse')
        tf1.reset_default_graph()
        return model

# Tuner definition
tuner = kt.RandomSearch(
    hypermodel=LSTMHyperModel(),
    objective='val_loss',
    max_trials=22,  # Number of hyperparameter combinations to try
    executions_per_trial=2,  # Number of models to train for each combination
    directory='my_dir',  # Directory where the tuner stores results
    project_name='lstm_without_normalize'
)


In [22]:
train_sequence = train[['center_x', 'center_y']].values

array([[0.372656  , 0.50625   ],
       [0.271094  , 0.0953125 ],
       [0.29335925, 0.18476575],
       ...,
       [0.23789075, 0.05      ],
       [0.24726575, 0.11289075],
       [0.105469  , 0.159375  ]])

In [25]:
temp = train_sequence.reshape(1, train_sequence.shape[0], train_sequence.shape[1])

In [29]:
temp

array([[[0.372656  , 0.50625   ],
        [0.271094  , 0.0953125 ],
        [0.29335925, 0.18476575],
        ...,
        [0.23789075, 0.05      ],
        [0.24726575, 0.11289075],
        [0.105469  , 0.159375  ]]])

In [ ]:
train[['center_x', 'center_y']].values

In [182]:
pred_X_normalizeds = scaler.transform(pred_sequence).reshape(1, pred_sequence.shape[0], pred_sequence.shape[1])

In [184]:
pred_X_normalizeds.shape

(1, 6, 2)

In [30]:
# Prepare training data
train_sequence = train[['center_x', 'center_y']].values
# scaler = MinMaxScaler()
train_X = train_sequence.reshape(1, train_sequence.shape[0], train_sequence.shape[1])

# Prepare test data
test_sequence = test[['center_x', 'center_y']].values
test_X = test_sequence.reshape(1, test_sequence.shape[0], test_sequence.shape[1])

train_y = train_X
test_y = test_X

# Hyperparameter search
tuner.search(train_X, train_y, validation_data=(test_X, test_y), epochs=100, batch_size=32)

# Get best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
print(best_hyperparameters.values)

# Evaluate the best model
test_loss = best_model.evaluate(test_X, test_y, verbose=0)
print(f"Test Loss: {test_loss}")

Trial 22 Complete [00h 00m 39s]
val_loss: 0.023133237846195698

Best val_loss So Far: 0.022336524911224842
Total elapsed time: 00h 25m 40s


c:\D\SKRIPSI\env\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 24 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Best hyperparameters:
{'num_layers': 3, 'units_0': 128, 'learning_rate': 0.004891738115501691, 'units_1': 512, 'units_2': 352}
Test Loss: 0.022219305858016014


In [31]:
best_model.save('lstm.keras')

In [33]:
from tensorflow import keras

In [34]:
# Specify the path to your saved model file
model_path = 'lstm.keras'  # Replace with your actual path

# Load the model architecture and weights
loaded_model = keras.models.load_model(model_path)

c:\D\SKRIPSI\env\lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 24 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [35]:
yolo_detection = {
    'x': [0.704687,
          0.510938,
          0.54453,
          0.078125,
          0.615625,
          0.167187],
    'y': [0.9875,
          0.125,
          0.277344,
          0.910156,
          0.575781,
          0.470313],
    'w': [0.040625,
          0.03125,
          0.0421875,
          0.078125,
          0.06875,
          0.053125],
    'h': [0.025,
          0.065625,
          0.0921875,
          0.179688,
          0.160938,
          0.134375]
}

In [36]:
df_pred = pd.DataFrame(yolo_detection)

In [37]:
df_pred['center_x'] = (df_pred['x'] + df_pred['w']) / 2
df_pred['center_y'] = (df_pred['y'] + df_pred['h']) / 2

In [45]:
df_pred

,x,y,w,h,center_x,center_y
0,0.704687,0.987500,0.040625,0.025000,0.372656,0.506250
1,0.510938,0.125000,0.031250,0.065625,0.271094,0.095312
2,0.544530,0.277344,0.042188,0.092188,0.293359,0.184766
3,0.078125,0.910156,0.078125,0.179688,0.078125,0.544922
4,0.615625,0.575781,0.068750,0.160938,0.342187,0.368360
5,0.167187,0.470313,0.053125,0.134375,0.110156,0.302344


In [174]:
pred_sequence = df_pred[['center_x', 'center_y']].values
pred_X = pred_sequence.reshape(1, feature_df.shape[0], feature_df.shape[1])

ValueError: cannot reshape array of size 12 into shape (1,861,2)

In [88]:
pred_X

array([[[0.372656  , 0.50625   ],
        [0.271094  , 0.0953125 ],
        [0.29335875, 0.18476575],
        [0.078125  , 0.544922  ],
        [0.3421875 , 0.3683595 ],
        [0.110156  , 0.302344  ]]])

In [103]:
predictions = loaded_model.predict(pred_X)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
[[0.27111426 0.13261217]]


In [143]:
temp_X = pred_sequence[[0]].reshape(1, train_sequence[[0]].shape[0], train_sequence[[i]].shape[1])
predictions = loaded_model.predict(pred_X[0][i])
rint(predictions)

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [175]:
for i in range (0,len(pred_X[0])):
    print(pred_X[0][i])

[0.372656 0.50625 ]
[0.271094  0.0953125]
[0.29335875 0.18476575]
[0.078125 0.544922]
[0.3421875 0.3683595]
[0.110156 0.302344]


In [97]:
pred_sequence[[i]].shape[0].pred

AttributeError: 'int' object has no attribute 'pred'

In [102]:
pred_sequence[[]].shape[0]

1

In [176]:
def res():
    result = []
    for i in range (0,5):
        pred_X = pred_sequence[[i]].reshape(1, pred_sequence[[i]].shape[0], pred_sequence[[i]].shape[1])
        predictions = loaded_model.predict(pred_X)
        print(predictions)
        result.append(predictions)
    return result

In [181]:
res = res()

TypeError: 'list' object is not callable

In [180]:
import cv2

for i in range(0,6):
    # Assuming you have an image (loaded using OpenCV) and bounding box information
    image = cv2.imread('C:\\D\\SKRIPSI\\yolov5\\runs\\detect\\inference_10\\images\\frame00001.png')  # Replace with your image loading method
    # x1, y1, x2, y2 = pred_x1,pred_y1, pred_x2, pred_y2 # Replace with your bounding box coordinates
    x, y, w, h =  res[i][0][0],res[i][0][1],df_pred.w[2],df_pred.h[2]# Replace with your bounding box coordinates
    # # Calculate width and height (if not provided)
    # width = x2 - x1
    # height = y2 - y1

    # # Draw the bounding box on the image
    # cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)  # Green color, thickness 2

    # # Display or save the image with the bounding box
    # cv2.imshow('Image with Bounding Box', image)
    # cv2.waitKey(0)  # Wait for a key press to close the window
    # cv2.destroyAllWindows()  # Clean up windows
    # image = cv2.imread('your_image.jpg')
    if image is not None:
        # Draw bounding box and display the image
        cv2.rectangle(image, (int(x*640), int(y*640)), (int((x+w)*640), int((y+h)*640)), (0, 255, 0), 2)
        cv2.imshow('Image with Bounding Box', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Error: Image could not be loaded!")

IndexError: list index out of range

In [ ]:
LSTM for prediction position of object